In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df  = pd.read_csv('/content/drive/MyDrive/secondary_data.csv', sep=';')

In [ ]:
df_1 = df.loc[df["class"] == "e"]
df_2 = df.loc[df["class"] == "p"]

In [ ]:
X_1 = df_1.drop("class", axis=1)
X_2 = df_2.drop("class", axis=1)
y_1 = df_1["class"]
y_2 = df_2["class"]

In [ ]:
from sklearn.model_selection import train_test_split

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1, y_1, test_size=0.15, random_state=87)
X_train_1, X_validation_1, y_train_1, y_validation_1 = train_test_split(X_train_1, y_train_1, test_size=0.18, random_state=87)

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2, test_size=0.15, random_state=87)
X_train_2, X_validation_2, y_train_2, y_validation_2 = train_test_split(X_train_2, y_train_2, test_size=0.18, random_state=87)

In [ ]:
X_train_inicial = pd.concat([X_train_1, X_train_2], ignore_index=True)
X_validation_inicial = pd.concat([X_validation_1, X_validation_2], ignore_index=True)
X_test_inicial = pd.concat([X_test_1, X_test_2], ignore_index=True)

y_train = pd.concat([y_train_1, y_train_2], ignore_index=True)
y_validation= pd.concat([y_validation_1, y_validation_2], ignore_index=True)
y_test = pd.concat([y_test_1, y_test_2], ignore_index=True)

#Encodings:

In [ ]:
#Para el target 1 si es e, 0 si es p

In [ ]:
y_train = y_train.map(lambda x: 1 if x == 'e' else 0)
y_validation = y_validation.map(lambda x: 1 if x == 'e' else 0)
y_test = y_test.map(lambda x: 1 if x == 'e' else 0)

In [ ]:
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import category_encoders as ce

In [ ]:
ohe_features = ce.OneHotEncoder(cols = ["does-bruise-or-bleed", "has-ring", "veil-type"], return_df=True)
X_train = ohe_features.fit_transform(X_train_inicial)
X_test = ohe_features.transform(X_test_inicial)
X_validation = ohe_features.transform(X_validation_inicial)

In [ ]:
te_features =  ce.TargetEncoder(cols = ["habitat", "season"], smoothing=2, return_df=True)
X_train = te_features.fit_transform(X_train, y_train)
X_test = te_features.transform(X_test, y_test)
X_validation = te_features.transform(X_validation, y_validation)

In [ ]:
be_features = ce.BinaryEncoder(cols = ["cap-shape", "cap-surface", "cap-color", "gill-attachment", "gill-spacing", "gill-color",
              "stem-color", "ring-type", "stem-root", "stem-surface", "veil-color", "spore-print-color"], return_df=True)
X_train = be_features.fit_transform(X_train)
X_test = be_features.transform(X_test)
X_validation = be_features.transform(X_validation)

In [ ]:
X_train[["cap-diameter", "stem-height", "stem-width"]].describe() #Normalizo y estandarizo estas columnas

,cap-diameter,stem-height,stem-width
count,42563.000000,42563.000000,42563.000000
mean,6.726122,6.572014,12.134153
std,5.230928,3.346977,9.975276
min,0.380000,0.000000,0.000000
25%,3.500000,4.640000,5.225000
50%,5.860000,5.950000,10.190000
75%,8.540000,7.740000,16.550000
max,59.460000,33.920000,98.000000


In [ ]:
cols = ["cap-diameter", "stem-height", "stem-width"]

for columna in cols:
  promedio = X_train[columna].mean()
  X_train[columna] = X_train[columna].map(lambda x: (x - X_train[columna].mean()) / X_train[columna].std())
  X_validation[columna] = X_validation[columna].map(lambda x: (x - X_validation[columna].mean()) / X_validation[columna].std())
  X_test[columna] = X_test[columna].map(lambda x: (x - X_test[columna].mean()) / X_test[columna].std())

In [ ]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Input

In [ ]:
from tensorflow.keras.optimizers import Adam

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
from tensorflow.keras.metrics import AUC

In [ ]:
from tensorflow.keras.utils import set_random_seed

In [ ]:
set_random_seed(10)

In [ ]:
model = Sequential([
    Dense(1000, activation="tanh", input_shape=(None,55)),
    Dense(100, activation="tanh"),
    Dense(10, activation="tanh"),
    Dense(1, activation="sigmoid")
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, None, 1000)        56000     
                                                                 
 dense_1 (Dense)             (None, None, 100)         100100    
                                                                 
 dense_2 (Dense)             (None, None, 10)          1010      
                                                                 
 dense_3 (Dense)             (None, None, 1)           11        
                                                                 
Total params: 157,121
Trainable params: 157,121
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=0.0001), metrics=[AUC(curve="ROC")])

In [ ]:
early_stopper = EarlyStopping(patience=5, restore_best_weights=True, monitor = "val_auc", min_delta= 0.001)

In [ ]:
model.fit(X_train,
          y_train,
          epochs=100,
          batch_size=300,
          callbacks=[early_stopper],
          validation_data=(X_validation, y_validation))

Epoch 1/100
142/142 [==============================] - 3s 11ms/step - loss: 0.5783 - auc: 0.7691 - val_loss: 0.5244 - val_auc: 0.8305
Epoch 2/100
142/142 [==============================] - 1s 9ms/step - loss: 0.4833 - auc: 0.8676 - val_loss: 0.4488 - val_auc: 0.8987
Epoch 3/100
142/142 [==============================] - 1s 8ms/step - loss: 0.4143 - auc: 0.9241 - val_loss: 0.3866 - val_auc: 0.9410
Epoch 4/100
142/142 [==============================] - 1s 9ms/step - loss: 0.3594 - auc: 0.9560 - val_loss: 0.3358 - val_auc: 0.9669
Epoch 5/100
142/142 [==============================] - 1s 9ms/step - loss: 0.3137 - auc: 0.9726 - val_loss: 0.2938 - val_auc: 0.9787
Epoch 6/100
142/142 [==============================] - 2s 12ms/step - loss: 0.2756 - auc: 0.9821 - val_loss: 0.2595 - val_auc: 0.9847
Epoch 7/100
142/142 [==============================] - 2s 13ms/step - loss: 0.2430 - auc: 0.9879 - val_loss: 0.2295 - val_auc: 0.9893
Epoch 8/100
142/142 [==============================] - 1s 8ms/step

In [ ]:
preds_validation = model.predict(X_validation)

292/292 [==============================] - 0s 1ms/step


In [ ]:
preds_test = model.predict(X_test)

287/287 [==============================] - 0s 1ms/step


In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(y_validation, preds_validation) #Su score en validacion es 0.9992798784478967

0.9992798784478967

In [ ]:
roc_auc_score(y_test, preds_test) #Su score en test es 0.999171303175798

0.999171303175798